In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'hh_ru_dataset.csv')

In [3]:
df.info()

In [4]:
df_cleared = df.copy()

In [5]:
df_cleared = df_cleared[(df_cleared['year_of_birth'] >= 1953) &
                        (df_cleared['year_of_birth'] <= 2005)]
len(df_cleared)

In [6]:
df_cleared = df_cleared.drop_duplicates(['topic_id'])
df_cleared = df_cleared[-((df.initial_state == 'discard') & (df.final_state == 'discard'))]
df_cleared = df_cleared[df.compensation_from <= df.compensation_to]
len(df_cleared)

In [7]:
df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state', 'final_state', 'work_schedule', 'compensation_from', 'compensation_to'])
len(df_cleared)

In [8]:
df_cleared = df_cleared[(2023 - df_cleared.work_experience_months / 12 - df_cleared.year_of_birth >= 18) & (df_cleared.work_experience_months <= 600)]
len(df_cleared)

In [9]:
quantile_param = 0.04
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['expected_salary'].quantile(quantile_param),
                       df_prof['expected_salary'].quantile(1 - quantile_param))

In [10]:
df_cleared.loc[:,'exp_salary_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'exp_salary_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]
len(df_cleared)

In [11]:
quantile_param = 0.04
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['compensation_from'].quantile(quantile_param),
                       df_prof['compensation_from'].quantile(1 - quantile_param))
df_cleared.loc[:,'compensation_from_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'compensation_from_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]
len(df_cleared)

In [12]:
quantile_param = 0.04
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['compensation_to'].quantile(quantile_param),
                       df_prof['compensation_to'].quantile(1 - quantile_param))
df_cleared.loc[:,'compensation_to_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'compensation_to_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]
len(df_cleared)

In [13]:
df_cleared = df_cleared[(df_cleared['expected_salary'] >= df_cleared['exp_salary_min_limit']) &
                        (df_cleared['expected_salary'] <= df_cleared['exp_salary_max_limit'])]
len(df_cleared)

In [14]:
df_cleared = df_cleared[(df_cleared['compensation_from'] >= df_cleared['compensation_from_min_limit']) &
                        (df_cleared['compensation_from'] <= df_cleared['compensation_from_max_limit'])]
len(df_cleared)

In [15]:
df_cleared = df_cleared[(df_cleared['compensation_to'] >= df_cleared['compensation_to_min_limit']) &
                        (df_cleared['compensation_to'] <= df_cleared['compensation_to_max_limit'])]
len(df_cleared)

In [16]:
df_cleared = df_cleared[df_cleared['compensation_from'] <= df_cleared['compensation_to']]

In [17]:
df_cleared['education_level'].unique()

In [18]:
gender_dict = {'female': 0, 'male': 1}
education_dict = {'secondary': 0, 'unfinished_higher': 1, 'special_secondary': 2,
                 'higher': 3, 'bachelor': 4, 'master': 5, 'candidate': 6, 'doctor': 7}
df_cleared = df_cleared.replace({'gender': gender_dict, 'education_level': education_dict})
df_cleared